In [51]:
import pandas as pd
import numpy as np
import requests
import os
import re
import csv
import plotly.express as px
import plotly.graph_objects as go
import math
import ast



In [46]:
selection = pd.read_csv("/media/hdd/tmp/bsp/meta.csv", dtype={'market_id': object, 'selection_id': object}, parse_dates = ['market_time'])

selection.traded_volume.value_counts()

0    25979
Name: traded_volume, dtype: int64

In [12]:
prices = pd.read_csv(
    "/media/hdd/tmp/bsp/prices.csv", 
    quoting=csv.QUOTE_ALL,
    dtype={'market_id': 'string', 'selection_id': 'string', 'atb_ladder': 'string', 'atl_ladder': 'string'},
    parse_dates=['time']
)
prices.head(3)

,market_id,selection_id,time,near_price,far_price,bsp_back_pool_stake,bsp_lay_pool_liability,atb_ladder,atl_ladder
0,1.183995724,22832649,2021-05-31 06:17:15.379,NaN,NaN,0,0,{},{}
1,1.183995724,13688381,2021-05-31 06:17:15.379,NaN,NaN,0,0,{},{}
2,1.183995724,2659734,2021-05-31 06:17:15.379,NaN,NaN,0,0,{},{}


In [13]:
prices.shape

(660111, 9)

In [14]:
prices.query('market_id == "1.183995724" and selection_id == "22832649"')

,market_id,selection_id,time,near_price,far_price,bsp_back_pool_stake,bsp_lay_pool_liability,atb_ladder,atl_ladder
0,1.183995724,22832649,2021-05-31 06:17:15.379,NaN,NaN,0,0,{},{}
10,1.183995724,22832649,2021-06-01 01:38:00.062,8.60,7.69,27,184,"{'p':[8,7.8,7.6,7.4,7.2],'v':[31.23,37.93,108....","{'p':[8.4,8.6,8.8,9,9.2],'v':[63.98,14.57,29.1..."
20,1.183995724,22832649,2021-06-01 01:38:10.158,8.60,7.69,27,184,"{'p':[8,7.8,7.6,7.4,7.2],'v':[25.54,49.14,113....","{'p':[8.2,8.4,8.6,8.8,9],'v':[37.63,68.03,41.2..."
30,1.183995724,22832649,2021-06-01 01:38:20.159,8.00,7.69,27,184,"{'p':[8,7.8,7.6,7.4,7.2],'v':[94.52,49.14,113....","{'p':[8.2,8.4,8.6,8.8,9],'v':[37.63,56.27,41.2..."
40,1.183995724,22832649,2021-06-01 01:38:30.182,8.00,7.69,27,190,"{'p':[8,7.8,7.6,7.4,7.2],'v':[62.88,49.14,113....","{'p':[8.2,8.4,8.6,8.8,9],'v':[22.99,56.27,41.2..."
50,1.183995724,22832649,2021-06-01 01:38:40.221,8.00,7.69,136,205,"{'p':[8.2,8,7.8,7.6,7.4],'v':[13.92,129.4,53.0...","{'p':[8.4,8.6,8.8,9,9.2],'v':[53.28,41.2,29.12..."
60,1.183995724,22832649,2021-06-01 01:38:50.923,8.00,7.69,136,205,"{'p':[8,7.8,7.6,7.4,7.2],'v':[129.4,53.01,133....","{'p':[8.4,8.6,8.8,9,9.2],'v':[67.76,41.2,29.12..."
70,1.183995724,22832649,2021-06-01 01:39:00.955,8.00,7.69,137,214,"{'p':[8.2,8,7.8,7.6,7.4],'v':[26.93,136.73,63....","{'p':[8.4,8.6,8.8,9,9.2],'v':[53.28,41.2,29.12..."
80,1.183995724,22832649,2021-06-01 01:39:10.962,8.00,7.69,137,229,"{'p':[8.2,8,7.8,7.6,7.4],'v':[29.2,146.88,61.3...","{'p':[8.4,8.6,8.8,9,9.2],'v':[53.26,23.52,32.3..."
90,1.183995724,22832649,2021-06-01 01:39:20.966,8.18,2.68,212,243,"{'p':[8.4,8.2,8,7.8,7.6],'v':[19.2,89.65,91.13...","{'p':[8.6,8.8,9,9.2,9.4],'v':[69.9,16.6,33.14,..."


In [75]:
# Join and augment
# prices['atb_ladder'] = [ast.literal_eval(x) for x in prices['atb_ladder']]
# prices['atl_ladder'] = [ast.literal_eval(x) for x in prices['atl_ladder']]

df = (
    selection.merge(prices, on = ['market_id', 'selection_id'])
    .assign(sbsj = lambda x: round((x['market_time'] - x['time']).dt.total_seconds() / 10) * 10)
    .assign(back_best = lambda x: [np.nan if d.get('p') is None else d.get('p')[0] for d in x['atb_ladder']])
    .assign(lay_best = lambda x: [np.nan if d.get('p') is None else d.get('p')[0] for d in x['atl_ladder']])
)

df.head(3)


,market_id,selection_id,venue,market_time,selection_name,traded_volume,win,bsp,time,near_price,far_price,bsp_back_pool_stake,bsp_lay_pool_liability,atb_ladder,atl_ladder,sbsj,back_best,lay_best
0,1.183995724,22832649,Hamilton,2021-06-01 01:40:00,1. Ablestock,0,0,8.4,2021-05-31 06:17:15.379,NaN,NaN,0,0,{},{},69760.0,NaN,NaN
1,1.183995724,22832649,Hamilton,2021-06-01 01:40:00,1. Ablestock,0,0,8.4,2021-06-01 01:38:00.062,8.6,7.69,27,184,"{'p': [8, 7.8, 7.6, 7.4, 7.2], 'v': [31.23, 37...","{'p': [8.4, 8.6, 8.8, 9, 9.2], 'v': [63.98, 14...",120.0,8.0,8.4
2,1.183995724,22832649,Hamilton,2021-06-01 01:40:00,1. Ablestock,0,0,8.4,2021-06-01 01:38:10.158,8.6,7.69,27,184,"{'p': [8, 7.8, 7.6, 7.4, 7.2], 'v': [25.54, 49...","{'p': [8.2, 8.4, 8.6, 8.8, 9], 'v': [37.63, 68...",110.0,8.0,8.2


# The BSP

- Volumes traded
- Accuracy
- Back profit / lay profit in certain odds ranges

In [ ]:
# Volume Traded
# ___________________________



In [77]:
df.groupby('sbsj', as_index = False).size().sort_values('size')

,sbsj,size
369,54220.0,4
691,76590.0,4
434,70800.0,4
559,74210.0,4
595,74800.0,4
...,...,...
78,30.0,25950
86,110.0,25950
81,60.0,25962
74,-10.0,26532


In [109]:
# Accuracy
# ___________________________

# Prepare dataframe with 

preplay = df[df.sbsj.isin([120,90,60,30,-10])][['market_id', 'selection_id', 'win', 'sbsj', 'back_best']].sort_values(['market_id', 'selection_id', 'sbsj'], ascending = [True, True, False]).rename(columns={'back_best': 'odds'}).assign(type = lambda x: "seconds before off: " + x['sbsj'].astype(str))
bsp = df.sort_values(['market_id', 'selection_id', 'time'], ascending = [True, True, False]).groupby(['market_id', 'selection_id']).head(1)[['market_id', 'selection_id', 'win', 'sbsj', 'bsp']].rename(columns={'bsp': 'odds'}).assign(type = "bsp")

accuracyFrame = pd.concat([preplay, bsp]).dropna()
accuracyFrame

,market_id,selection_id,win,sbsj,odds,type
280770,1.181289390,2004182,0,120.0,3.85,seconds before off: 120.0
280773,1.181289390,2004182,0,90.0,3.80,seconds before off: 90.0
280776,1.181289390,2004182,0,60.0,3.80,seconds before off: 60.0
280779,1.181289390,2004182,0,30.0,3.90,seconds before off: 30.0
280783,1.181289390,2004182,0,-10.0,3.80,seconds before off: -10.0
...,...,...,...,...,...,...
6594,1.184845396,24644026,0,-20.0,39.58,bsp
6662,1.184845396,28640078,0,-20.0,13.24,bsp
6645,1.184845396,37583926,0,-20.0,70.42,bsp
6628,1.184845396,37996263,1,-20.0,5.97,bsp


In [110]:
# Implied chance vs Actual chance ++++++++++++++++++++++++++++++++++++++++

# BSP Only
# __________________

winRates = (
    accuracyFrame
    .query('type == "bsp"')
    .assign(implied_chance = lambda x: round(20 * (1 / x['odds']))/20)
    .groupby('implied_chance', as_index = False)
    .agg({'win': 'mean'})
)

fig = px.scatter(winRates, x = "implied_chance", y = "win", template = "plotly_white", title = "BSP: implied win vs actual win")
fig.add_trace(
    go.Scatter(
        x = winRates.implied_chance, y = winRates.implied_chance, name = 'no bias', line_color = 'rgba(8,61,119, 0.3)'
    )
)
fig.show()

In [111]:
# Odds Comparison
# __________________

winRates = (
    accuracyFrame
    .assign(implied_chance = lambda x: round(20 * (1 / x['odds']))/20)
    .groupby(['type', 'implied_chance'], as_index = False)
    .agg({'win': 'mean'})
)

fig = px.scatter(winRates, x = "implied_chance", y = "win", color = 'type', template = "plotly_white", title = "Comparing Price Points: implied win vs actual win")
fig.add_trace(
    go.Scatter(
        x = winRates.implied_chance, y = winRates.implied_chance, name = 'no bias', line_color = 'rgba(8,61,119, 0.3)'
    )
)
fig.show()


In [112]:
# Logloss ++++++++++++++++++++++++++++++++++++++++

from sklearn.metrics import log_loss

# Overall Logloss
# _________________

bspLoss = log_loss(
    y_pred = 1 / accuracyFrame.query('type == "bsp"')['odds'],
    y_true = accuracyFrame.query('type == "bsp"')['win']
)

print(f'The overall logloss of the BSP is {round(bspLoss,4)}')

# Logloss at Different Time Points
# _________________

accuracyFrame.groupby('type').apply(lambda x: log_loss(y_pred=1/x['odds'],y_true=x['win']))


# TODO
#   Replace the preplay odds best_back with a midpoint between best back and lay
#   Cause I shouldn't be getting lower logloss on the preplay price vs the bsp price it kind of undermines my whole point here



The overall logloss of the BSP is 0.2767


type
bsp                          0.276721
seconds before off: -10.0    0.275442
seconds before off: 120.0    0.276079
seconds before off: 30.0     0.275537
seconds before off: 60.0     0.275786
seconds before off: 90.0     0.275907
dtype: float64

In [107]:
accuracyFrame.sort_values('odds')

,market_id,selection_id,win,sbsj,odds,type
349005,1.181608776,28364729,1,-10.0,1.03,seconds before off: -10.0
349005,1.181608776,28364729,1,-10.0,1.09,bsp
465822,1.182079391,37793933,1,-20.0,1.12,bsp
465807,1.182079391,37793933,1,120.0,1.12,seconds before off: 120.0
465810,1.182079391,37793933,1,90.0,1.13,seconds before off: 90.0
...,...,...,...,...,...,...
257178,1.184497290,39880373,0,120.0,NaN,seconds before off: 120.0
257196,1.184497290,39880374,0,120.0,NaN,seconds before off: 120.0
257214,1.184497290,39880375,1,120.0,NaN,seconds before off: 120.0
257250,1.184497290,39880377,0,120.0,NaN,seconds before off: 120.0


# Projecting BSP

- Near price
    + showing 60 second lag
    +